# Quantum Variational Monte Carlo Ising model in 1D using Simulated Annealing

In this notebook we will try to find the optimal parameters of a trial wavefunction describing the 1D quantum Ising model for bosons with varying number of spins. The trial wavefunction has an RBM-like expression:
$$
    \Psi(\mathcal{S}) = e^{b^Ts}\prod_{i=1}^{n_h}2\cosh(c_i + W^T_is)
$$
where $n_h$ is the number of hidden units. For bosons, we require some additional conditions that the wavefunction must fulfill, specifically the symmetry under the exchange of any pair of spins requires that all visible bias must be the same $b_j = b,\,j=1,\ldots,n_v$ and the weights connecting the visible layer to each hidden unit must also be equal to each other $\omega_{ij} = \omega_i,\,j=1,\ldots,n_h$. With this, the previous expression changes to
$$
    \Psi(\mathcal{S}) = e^{b\sum_js_j}\prod_{i=1}^{n_h}2\cosh(c_i + w_i\sum_{j=1}^{n_v}s_j)
$$
which simplifies a bit the exploration of parameters. According to the variational method, this wavefunction will provide an upper bound to the ground state energy that we are trying to estimate:
$$
    E = \frac{\langle\Psi|\hat{H}|\Psi\rangle}{\langle\Psi|\Psi\rangle}
$$
where the Hamiltonian of the Transverse Field Ising model is:
$$
    \hat{H} = -J\sum_{i=1}^{N-1}\hat{\sigma}^z_i\hat{\sigma}^z_{i+1} -B\sum_{j=1}^N\hat{\sigma}^x_j
$$
$\hat{\sigma}^z,\hat{\sigma}^x$ being the Pauli matrices acting on the spins, $J,B$ the coupling and the transverse magnetic field term, respectively.

## Simulated Annealing

In order to find the optimal configuration of parameters which minimizes the energy, we will use a simulated annealing method: 
 - Starting from a random initial configuration of the parameters.
 - Proposing a new set of parameters $\theta\to\theta'$.
 - Accepting the change with probability $P(\theta')/P(\theta)$. Where the PDF is $P(\theta) = \text{exp}(E(\theta)/T)$, $T$ being a fictitious temperature.
 - Cooling down the temperature by a factor $\gamma$ each iteration: $T := T\times\gamma$.

### Libraries

In [1]:
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import matplotlib.pyplot as plt
from RBM import RBM
import Ising1D as tfi

### Simulated annealing process

In [2]:
def simulatedAnnealing(rbm,H,N_iter=1000,thermalise=10,T0=10.0,gamma=0.999,dt=0.1):    
    # Initialize some variables
    current_solution = deepcopy(rbm)
    psi = current_solution.wavefunction()
    current_energy = psi.conj().T @ H @ psi
    
    # Temperature
    T = T0

    # Keep track of acceptance rate
    accepts = 0

    for iter in tqdm(range(N_iter)):
        for _ in range(thermalise):
            displacement = np.random.normal(scale=dt,size=current_solution.params.shape)
            current_solution.params += displacement
        
            # Calculate the difference in energies
            psi = current_solution.wavefunction()
            delta_energy = psi.conj().T @ H @ psi - current_energy
    
            # Accept new solution based on probability of acceptance (or its logarithm)
            if delta_energy < 0 or -delta_energy/T > np.log(np.random.rand()):
                current_energy += delta_energy
                accepts += 1
            else:
                current_solution.params -= displacement

        # Cooldown temperature
        T *= gamma
    
    # At the end of the algorithm, set the rbm parameters to the solution
    rbm.params = current_solution.params

    # Print acceptance probability
    print("Acceptance probability:",accepts/(N_iter*thermalise))
    
    # Return the minimum energy
    return current_energy

### Test

In [3]:
# Random seed
np.random.seed(7)

# Initialize parameters
N = 5 # Number of spins
h = 2 # Number of hidden units
J,B = 2,1
N_iter = 10000 # Number of iterations of simulated annealing

# Initialize Hamiltonian and rbm
H = tfi.buildHamiltonian_sparse(N,J,B)

rbm = RBM(N,h,sigma=0.1)

# Start simulated annealing
best_energy = simulatedAnnealing(rbm,H,N_iter=N_iter,thermalise=1,T0=20.0,gamma=0.999,dt=0.01)

# Compare with real ground state energy
ground_energy, ground_state = tfi.diagonalizeHamiltonian_sparse(H)

print("Energy: %.6f" % np.real(best_energy))
print("Error: %.5f" % np.real((ground_energy - best_energy)/ground_energy*100))

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 818.73it/s]

Acceptance probability: 0.9313
Energy: -10.599434
Error: 0.48121
